![Podmanlogo](Pictures/podman-logo.png)

# Podman ecosystem

We have seen what an awsome piece of software Podman is, but that is not all. Podman counts with some other tools in its ecosystem that makes everything run smoothly.
Lets review all of them.

## Managing multiple containers: podman compose and kubernetes

Running a single container with podman is just fine, we have demonstrated how simple it is througout this workshop. But, what happens if I want to run multiple containers simultaneously? Do you need to execute them one by one?

You can use the tool podman compose to execute multiple containers in a single command. Podman compose is a tool that helps you defining the list of containers and all of their configuration in a text file written in yaml format. Then you can choose to execute, stop, remove, update and so on all of the containers defined in the file at once. Podman compose is a very powerful tool but it's limited to a single system.

If you're looking at deploying your containers in multiple systems simultaneously in order to get high availability then you will most probably end up using kubernetes as it has become the standard platform for running containerized workloads in the enterprise.
Kubernetes, same as podman composer, uses yaml files to define the desired state of your containerized workloads and, once you pass the file to the control plane nodes of kubernetes, it will make sure your containers run following your specifications and they will be able to run in any of the nodes that are part of your kubernetes cluster.

![k8slogo](Pictures/kubernetes-logo.png)

Now, transitioning from single system container engines like Podman or Docker to kubernetes may not always be easy. That is why Podman can generate a kubernetes yaml file from the container that you are running righ now with Podman. First you just need to execute a new container:

In [ ]:
!podman run -d --rm --name=kubepodman docker.io/redhat/ubi9 sleep 999

Now your container is running:

In [ ]:
!podman ps

With this, you can execute the "podman generate kube" command to export a yaml file that then you can use for your kubernetes environment.

In [ ]:
!podman generate kube kubepodman > kubepodman.yml
!cat kubepodman.yml

The output of the previous command is the content of the yaml file that you need to use for running your container in a kubernetes workload. At the beginning of the file you can even see a few comments in which you find the command you need to run in order to start executing this container in your kubernetes cluster: "kubectl create -f kubepodman.yml".

As you can see, podman has multiple tools that allow you to transition to managing multiple containers at once and simplify the transition to those tools.

## Advanced container images management: Skopeo

Skopeo is a tool for manipulating, inspecting, signing, and transferring container images and image repositories on linux systems, Windows and MacOS. Like Podman and Buildah, Skopeo is an open source community-driven project that does not require running a container daemon.

With Skopeo, you can inspect images on a remote registry without having to download the entire image with all its layers, making it a lightweight and modular solution for working with container images across different formats, including Open Container Initiative (OCI) and Docker images.

![skopeologo](Pictures/skopeo.png)

In most cases you will find Skopeo a super useful tool to manage container images that are stored in remote registries without pulling them to your system. For example you can inspect a remote container by using the command "skopeo inspect". First lets make sure that the image we will be inspecting is not locally stored in our system.

In [24]:
!podman images

REPOSITORY                                                          TAG         IMAGE ID      CREATED        SIZE
<none>                                                              <none>      95f0b9bd1c87  24 hours ago   314 MB
quay.io/skupper/skupper-router                                      2.4.3       9f6d24d58351  10 days ago    275 MB
localhost/bcl-ov                                                    15          ef3ba4fe4c30  6 weeks ago    717 MB
<none>                                                              <none>      2be670a5b439  6 weeks ago    803 MB
<none>                                                              <none>      7a4c1f405114  6 weeks ago    369 MB
quay.io/mschreie/bcl-ov                                             14          a3e1ce7d79a7  2 months ago   717 MB
<none>                                                              <none>      af6827d3105f  2 months ago   299 MB
quay.io/redhat_emp1/ee-ansible-ssa                                  latest

You should see an empty list.

Now lets inspect a remote container image, an image residing in docker.io, with skopeo:

In [25]:
!skopeo inspect docker://docker.io/redhat/ubi9-minimal

{
    "Name": "docker.io/redhat/ubi9-minimal",
    "Digest": "sha256:c8c7a06ce1c5fa23c1cbd7a0fd891eacd099bc232aa9985ddb183cfe98d1deaf",
    "RepoTags": [
        "9.0.0",
        "9.0.0-1471",
        "9.0.0-1471-source",
        "9.0.0-1471.1655190711",
        "9.0.0-1471.1655190711-source",
        "9.0.0-1575",
        "9.0.0-1575-source",
        "9.0.0-1580",
        "9.0.0-1580-source",
        "9.0.0-1608",
        "9.0.0-1608-source",
        "9.0.0-1644",
        "9.0.0-1644-source",
        "9.0.0-1644.1666621587",
        "9.0.0-1644.1666621587-source",
        "9.0.0-1687",
        "9.0.0-1687-source",
        "9.0.0-1700",
        "9.0.0-1700-source",
        "9.1",
        "9.1.0",
        "9.1.0-1656",
        "9.1.0-1656-source",
        "9.1.0-1656.1669627757",
        "9.1.0-1656.1669627757-source",
        "9.1.0-1760",
        "9.1.0-1760-source",
        "9.1.0-1760.1675784957",
        "9.1.0-1760.1675784957-source",
        "9.1.0-1793",
        "9.1.0-1793-sour

Pay attention to the naming convention, you need to specify "docker://" before the repo, container name and tag.

As you can see we are able to get the information about the container image but we did not need to pull the image for that. Lets check that the image was not downloaded.

In [26]:
!podman images

REPOSITORY                                                          TAG         IMAGE ID      CREATED        SIZE
<none>                                                              <none>      95f0b9bd1c87  24 hours ago   314 MB
quay.io/skupper/skupper-router                                      2.4.3       9f6d24d58351  10 days ago    275 MB
localhost/bcl-ov                                                    15          ef3ba4fe4c30  6 weeks ago    717 MB
<none>                                                              <none>      2be670a5b439  6 weeks ago    803 MB
<none>                                                              <none>      7a4c1f405114  6 weeks ago    369 MB
quay.io/mschreie/bcl-ov                                             14          a3e1ce7d79a7  2 months ago   717 MB
<none>                                                              <none>      af6827d3105f  2 months ago   299 MB
quay.io/redhat_emp1/ee-ansible-ssa                                  latest

Another very intereting and widely used skopeo functionality is copying images from a source registry to a target registry with the "skopeo copy" command, by using it you don't need to have a copy of the image in your local system.

## Advanced container image creation using buildah

The command line tool buildah provides the hability of creating container images from a running container, a Containerfile or from scratch. The resulting images are OCI (Open Container Initiative) compliant, meaning that they will work with any container enginge that complies with the OCI standard such as podman, docker or CRI-O.

![buildahlogo](Pictures/buildah.png)

Most enterprises will be able to just work with Podman build capabilities, but if you need to cover an advanced use case like building a container image from scratch, that is without a base container image, buildah is definitely the tool you're looking for. We will not cover this use case in this workshop as it's quite advanced.

## Podman for Desktop

If you're not experienced with command line tools there is a super powerful tool that brings a graphical user experience to Podman, that is Podman Desktop. It is available for Windows, macOS and Linux.

![PodmanDesktop](Pictures/Podman_Desktop.png)

Moreover, Podman Desktop is not limited to Podman, it can also interact with kubernetes clusters and is a very helpful tool in the transition to containerized applications for enterprises that are willing to take this path.

# Cleanup

In [27]:
!podman stop --all
!podman rm --all
!podman rmi docker.io/redhat/ubi9
!rm kubepodman.yml

Error: docker.io/redhat/ubi9: image not known
